In [6]:
#importações
import pandas as pd
import numpy as np
import re
import string
#o que é
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download the missing resource
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [16]:
#carregando e visualisando o dataset
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
print(f"\nCorpo do dataset (linhas/colunas): {df.shape}\n")
df.head(10)




Corpo do dataset (linhas/colunas): (50000, 2)



,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [17]:
#Analise exploratoria
df.info()
df.describe()
print()

"""
Nenhuma linha faltando
Sem valores nulos
object = strings, rótulos 'positive' ou 'negative', nenhum tipo numérico — dados de texto (NLP) #PESQUISAR
50000 avaliações (49582 avaliaçoes unicas e 418 duplicadas)
top = texto mais repetido, (5 vezes)
freq = 25.000 positivos (balanceado)

"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB



"\nNenhuma linha faltando\nSem valores nulos\nobject = strings, rótulos 'positive' ou 'negative', nenhum tipo numérico — dados de texto (NLP) #PESQUISAR\n50000 avaliações (49582 avaliaçoes unicas e 418 duplicadas)\ntop = texto mais repetido, (5 vezes)\nfreq = 25.000 positivos (balanceado)\n\n"

In [18]:
#limpeza e processamento

df = df.drop_duplicates() # remover as duplicadas
print(f"Após remoção de duplicatas: {df.shape}")

# Particao para teste DEPOIS EXCLUIR
df = df.sample(2000, random_state=50)


stop = set(stopwords.words('english')) #conjunto com todas as palavras irrelevantes (stopwords) em inglês como "the", "is", "and" etc.
lem = WordNetLemmatizer()

#função que faz o processamento do texto
def processar(text):
    text = text.lower() #normalizar o texto em minuscula
    text = re.sub(r"<.*?>", "", text) #remover tags html
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text) #remover pontuações
    tokens = word_tokenize(text) #tokenizar o texto em palavras
    tokens = [t for t in tokens if t not in stop and t.isalpha()] #lista de tokens filtrados
    tokens = [lem.lemmatize(t) for t in tokens] #Lematiza cada token
    return " ".join(tokens) #junta e retorna uma unica string

# Aplicar ao dataset
df['clean_review'] = df['review'].apply(processar)
#compara o antes e depois da limpeza
df[['review', 'clean_review']].head()



Após remoção de duplicatas: (49582, 2)


,review,clean_review
25064,A touching documentary that puts a human face ...,touching documentary put human face tragedy sh...
10548,A doctor who is trying to complete the medical...,doctor trying complete medical dream transplan...
5284,"After having seen and loved Postal (yes, I act...",seen loved postal yes actually loved postal de...
45293,Jack Black and Kyle Gass team together to prom...,jack black kyle gas team together promote band...
37372,I can only assume the previous posts came from...,assume previous post came exec production comp...


In [ ]:
#VETORIZAÇÃO
X_treino, X_teste, y_treino, y_teste = train_test_split(df['clean_review'], df['sentiment'], test_size=0.3, random_state=50) #treinamento(70/30)

vetorizar = TfidfVectorizer() #vetorizar com  TF-IDF, mais peso as palavras frequentes no documento
X_treino_vet = vetorizar.fit_transform(X_treino)
X_teste_vet = vetorizar.transform(X_teste)

#transforma os rotulos em 0 ou 1
y_treino = y_treino.map({'positive': 1, 'negative': 0})
y_teste = y_teste.map({'positive': 1, 'negative': 0})


"""amostra da vetorização
print(X_treino_vet)
print(X_teste_vet)"""




In [ ]:
#graficos e metricas
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#grafico curva roc
def plotar_curva_roc(modelo, X_teste, y_teste, nome_modelo):
    # Usa o método correto para obter probabilidades
    if hasattr(modelo, "predict_proba"):
        y_proba = modelo.predict_proba(X_teste)[:, 1]
    elif hasattr(modelo, "prever_prob"):
        y_proba = modelo.prever_prob(X_teste)
    else:
        raise AttributeError("O modelo não possui um método de probabilidade conhecido.")

    fpr, tpr, _ = roc_curve(y_teste, y_proba)
    auc = roc_auc_score(y_teste, y_proba)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("Falsos Positivos (FPR)")
    plt.ylabel("Verdadeiros Positivos (TPR)")
    plt.title(f"Curva ROC - {nome_modelo}")
    plt.legend()
    plt.grid(True)
    plt.show()

#grafico matriz de confusao

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plotar_matriz_confusao(y_true, y_pred, nome_modelo):
    matriz = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Negativo', 'Positivo'],
                yticklabels=['Negativo', 'Positivo'])
    plt.title(f"Matriz de Confusão - {nome_modelo}")
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()




In [ ]:
#modelo arvore e svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix #metricas de avaliação

# Treinar árvore de decisão
modelo_arv = DecisionTreeClassifier() #modelo
modelo_arv.fit(X_treino_vet, y_treino) #aprende com os dados de treino
pred_arv = modelo_arv.predict(X_teste_vet) #prever

# Treinar SVM
svm = SVC(probability=True)
svm.fit(X_treino_vet, y_treino)
pred_svm = svm.predict(X_teste_vet)

# Avaliação
def avaliar_modelo(nome, y_true, y_pred):
    print(f"\n--- {nome} ---")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.2%}")
    print(f"Precisão: {precision_score(y_true, y_pred):.2%}")
    print(f"Recall: {recall_score(y_true, y_pred):.2%}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.2%}")
    print()

avaliar_modelo("Árvore de Decisão", y_teste, pred_arv)
avaliar_modelo("SVM", y_teste, pred_svm)
plotar_matriz_confusao(y_teste, pred_arv, "Árvore de Decisão")
plotar_curva_roc(modelo_arv, X_teste_vet, y_teste, "Árvore de Decisão")
print()
plotar_matriz_confusao(y_teste, pred_svm, "SVM")
plotar_curva_roc(svm, X_teste_vet, y_teste, "SVM")

In [ ]:
#MODELO REDE NEURAL
from sklearn.preprocessing import StandardScaler

#Funções
#saida 0 ou 1
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def derivada_sigmoid(a):
   return a * (1 - a)

# Rede Neural 1 camada oculta
class RedeMLP:
    #tres camadas(entrada,ocukta,saida)
    def __init__(self, n_atrb, n_neuronio, taxa_aprend=0.01, epocas=5000,
                 random_state=42):
        rng = np.random.default_rng(random_state)

        self.pesosO = rng.normal(0, np.sqrt(2/n_atrb), size=(n_atrb, n_neuronio))
        self.biasO = np.zeros(n_neuronio)
        self.pesosS = rng.normal(0, np.sqrt(2/n_neuronio), size=(n_neuronio, 1))
        self.biasS = np.zeros(1)
        self.lr = taxa_aprend
        self.epocas = epocas

    #propagation
    def _forward(self, X):
        z1 = X @ self.pesosO + self.biasO
        a1 = sigmoid(z1)    #ativar camada oculta
        z2 = a1 @ self.pesosS + self.biasS
        a2 = sigmoid(z2)    # ativar camada saída
        return z1, a1, z2, a2

    #Backpropagation
    #x=entrada, y=rotulos
    def _backward(self, X, y, z1, a1, a2):
        m = X.shape[0]
        erro_saida = a2.squeeze() - y       # usar para corrigir os pesos da última camada.
        grad_pesosS = (a1.T @ erro_saida[:, None]) / m
        grad_biasS = np.mean(erro_saida)
        erro_oculta = (erro_saida[:, None] @ self.pesosS.T) * derivada_sigmoid(a1) #como o erro da saida afeta a camda oculta
        grad_pesosO = (X.T @ erro_oculta) / m
        grad_biasO = np.mean(erro_oculta, axis=0)
        # Atualização
        self.pesosO -= self.lr * grad_pesosO
        self.biasO -= self.lr * grad_biasO
        self.pesosS -= self.lr * grad_pesosS
        self.biasS -= self.lr * grad_biasS

    def treinar(self, X, y):
        for _ in range(self.epocas):
            z1, a1, z2, a2 = self._forward(X)
            self._backward(X, y, z1, a1, a2)

    def prever_prob(self, X):
        _, _, _, a2 = self._forward(X)
        return a2.ravel()

    def prever(self, X, thresh=0.5):
        return (self.prever_prob(X) >= thresh).astype(int)


X = vetorizar.transform(df['clean_review']).toarray()
y = df['sentiment'].map({'positive': 1, 'negative': 0}).values

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
scaler = StandardScaler()
X_treino = scaler.fit_transform(X_treino)
X_teste = scaler.transform(X_teste)
#Treinamento
config_oculta = [1]
for n_neuronio in config_oculta:
    rede = RedeMLP(n_atrb=X.shape[1], n_neuronio=n_neuronio, taxa_aprend=0.01, epocas=5000)
    rede.treinar(X_treino, y_treino)

    y_pred = rede.prever(X_teste)
    acuracia = np.mean(y_pred == y_teste)
    print(f"Camada oculta: {n_neuronio:>2} neurônios → Acurácia: {acuracia:.2%}")

avaliar_modelo("Rede Neural (1 camada oculta)", y_teste, y_pred)
plotar_matriz_confusao(y_teste, y_pred, "Rede Neural (1 camada oculta)")
print()
plotar_curva_roc(rede, X_teste, y_teste, "Rede Neural (1 camada oculta)")



In [ ]:
#modelo KNN
from sklearn.neighbors import KNeighborsClassifier

# Treinamento
modelo_knn = KNeighborsClassifier(n_neighbors=55)
modelo_knn.fit(X_treino_vet, y_treino)

# Previsão
y_pred_knn = modelo_knn.predict(X_teste_vet)

# Avaliação
avaliar_modelo("KNN (k=5)", y_teste, y_pred_knn)
plotar_matriz_confusao(y_teste, y_pred_knn,"Matriz de Confusão - KNN")
print()
plotar_curva_roc(modelo_knn, X_teste_vet, y_teste, "Curva ROC - KNN")

In [ ]:
#kmeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


kmeans = KMeans(n_clusters=2, random_state=50)
clusters = kmeans.fit_predict(X_teste_vet)

print("\nTabela cruzada (cluster x rótulo real):")
display(pd.crosstab(clusters, y_teste))

#avaliaçoes e grafico
# Silhouette Score mede o quão bem separados estão os clusters
score = silhouette_score(X_teste_vet, clusters)
print(f"\nSilhouette Score do K-Means: {score:.4f}")

clusters_mapeados = np.where(clusters == 0, 0, 1)
avaliar_modelo("Modelo Kmeans", y_teste, clusters_mapeados)
plotar_matriz_confusao(y_teste, clusters_mapeados,"Matriz de Confusão - Kmeans")







In [ ]:
# Escolher o melhor modelo
resultados_modelos = {}

def registrar_resultado(nome, y_true, y_pred, proba=None):
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, proba if proba is not None else y_pred)

    resultados_modelos[nome] = {
        'Acurácia': acc,
        'Precisão': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc
    }
# Árvore
registrar_resultado("Árvore de Decisão", y_teste, pred_arv, modelo_arv.predict_proba(X_teste_vet)[:, 1])

# SVM
registrar_resultado("SVM", y_teste, pred_svm, svm.predict_proba(X_teste_vet)[:, 1])

# Rede Neural
registrar_resultado("Rede Neural", y_teste, y_pred, rede.prever_prob(X_teste))

# KNN
registrar_resultado("KNN", y_teste, y_pred_knn, modelo_knn.predict_proba(X_teste_vet)[:, 1])

# K-Means
registrar_resultado("K-Means", y_teste, clusters_mapeados)

# Agente que escolhe o melhor modelo
def escolher_melhor_modelo(metricas=["F1-Score","Acurácia", "Precisão","AUC"]):
    print("\n Ranking dos Modelos:")
    df_resultados = pd.DataFrame(resultados_modelos).T
    print(df_resultados.sort_values(by=metricas, ascending=False))

    melhor = df_resultados.sort_values(by=metricas, ascending=False).index[0]
    print(f"\n Melhor modelo de acordo com {', '.join(metricas)}: {melhor}")
    return melhor
melhor_modelo = escolher_melhor_modelo(metricas=["F1-Score","Acurácia", "Precisão","AUC"])
